In [1]:
# essential
#functions.py holds all self-written f, so that this is merely execution/testing

from functions import *

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import re

In [2]:
# essential
#load and preprocess data

df = load_data('data', 'test_flow.csv', wide_boolean=False, idx_boolean=False)

In [12]:
#optional, if desiring to visualize statically without interactivity
#need to set year and unit to filter dataframe

#filter payload
unit = 'TJ'
year = 2017

#filtering df
filter_list = [i and j for i, j in zip(df['Value.type'] == unit, df['Value.info'] == year)]
temp_df = df[filter_list]

#creating ordered lists of nodes' indices and their x-levels
elements_positions, unique_sources_targets = calc_node_x(temp_df, 'Source', 'Target')

#redeclare node_dict (this is done, because filtering the same node_dict will not work with Plotly as no breaks in numeric/integer node sequences can be handled. An alternative may be to manipulate the broken sequence, but manage to access the node_dict keys anyway)
node_dict = dict()
for i,e in zip(range(len(unique_sources_targets)), elements_positions):
    node_dict[str(unique_sources_targets[i])] = {'Source_level_str': e, 'Source_level_int': x_lvl_dict[e]["Int"], 'Node_index': i}

#save each node's aggregated values in node_dict
aggregate_node_values(temp_df, node_dict)
#save each x-level's aggregated values in x_lvl_dict
aggregate_values_by_x_level(node_dict, x_lvl_dict)

payload_x, payload_y = nodes_xypositions(elements_positions, node_dict, x_lvl_dict)

#sankey source and target payload for link dict
source, target = calculate_link_sources_targets_alt(temp_df, node_dict)

# static sankey visualization without interactivity
fig = go.Figure(data=[go.Sankey(
    arrangement= "freeform",
    node = dict(
      label = payload_nodes,
      x = [.001 if x==0.2 else round(x - 0.2,1) for x in payload_x],
      y = [.999 if y==1 else y for y in payload_y],
      pad = 8,
      thickness = 5,
      color = "cornsilk"
    ),
    link = dict(
      source = source,
      target = target,
      value = temp_df['Value']/1000,
      label = temp_df['Label'],
      color = temp_df['Colour']
  ))])

fig.update_layout(title_text=f"EU27 material flows in {year} for {unit} along Sourcing | Availability | Processing | Services/goods | Outflows", font_size=10)
# fig.update_layout(width = 10*1920/10, height = 7*1080/10, title_text="EU28 material flows in 2017 Mass_now/kt Source | Availability | Processing | Services/goods | Outflows", font_size=10)
fig.show()

In [18]:
#essential
#dynamic sankey visualization with radio buttons and slider filters
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div(children=[
            html.Div(id='my-output')
        ], style={'margin': 'auto', 'text-align': 'center', 'fontFamily': 'Arial'}),
        html.Br(),
        html.Br(),
        html.Div([
            dcc.RadioItems(
                df['Value.type'].unique(),
                value = df['Value.type'].unique()[0],
                id='xaxis-column',
                inline=True
            )], style={'margin': 'auto', 'fontFamily': 'Arial', 'width': '48%','text-align': 'center'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    html.Div([
        dcc.Slider(
            df['Value.info'].min(),
            df['Value.info'].max(),
            step=None,
            id='year--slider',
            value=df['Value.info'].max(),
            marks={str(year): str(year) for year in df['Value.info'].unique()},
        )], style={'fontFamily': 'Arial', 'width': '70%', 'margin': 'auto'})
    
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    Output('my-output', 'children'),
    Input('xaxis-column', 'value'),
    Input('year--slider', 'value'))

def update_graph_and_title(xaxis_column_name, year_value):
    
    filter_list = [i and j for i, j in zip(df['Value.type'] == xaxis_column_name, df['Value.info'] == year_value)]
    temp_df = df[filter_list]
    
    #creating ordered lists of nodes' indices and their x-levels
    elements_positions, unique_sources_targets = calc_node_x(temp_df, 'Source', 'Target')

    #redeclare node_dict (this is done, because filtering the same node_dict will not work with Plotly as no breaks in numeric/integer node sequences can be handled. An alternative may be to manipulate the broken sequence, but manage to access the node_dict keys anyway)
    node_dict = dict()
    for i,e in zip(range(len(unique_sources_targets)), elements_positions):
        node_dict[str(unique_sources_targets[i])] = {'Source_level_str': e, 'Source_level_int': x_lvl_dict[e]["Int"], 'Node_index': i}

    #save each node's aggregated values in node_dict
    aggregate_node_values(temp_df, node_dict)
    #save each x-level's aggregated values in x_lvl_dict
    aggregate_values_by_x_level(node_dict, x_lvl_dict)

    payload_x, payload_y = nodes_xypositions(elements_positions, node_dict, x_lvl_dict)

    #sankey source and target payload for link dict
    source, target = calculate_link_sources_targets_alt(temp_df, node_dict)
    
    payload_title = f"EU27 material flows in {year_value} for {xaxis_column_name} along Sourcing | Availability | Processing | Services/goods | Outflows"

    new_fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        arrangement= "freeform",
        node = dict(
          label = payload_nodes,
          x = [.001 if x==0.2 else round(x - 0.2,1) for x in payload_x],
          y = [.999 if y==1 else y for y in payload_y],
          pad = 8,
          thickness = 5,
          color = "cornsilk"
        ),
        link = dict(
          source = source,
          target = target,
          value = temp_df['Value']/1000,
          label = temp_df['Label'],
          color = temp_df['Colour']
      ))])
    
    new_fig.update_layout(height=700)

    return new_fig, payload_title
if __name__ == '__main__':
    app.run_server(port = 8052, mode = "inline", debug=True)